我们已经从上一节里了解了多层感知机的原理。下面，我们一起来动手实现一个多层感知机。首先导入实现所需的包或模块。

In [2]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("..") # 为了导入上层目录的d2lzh_tensorflow
import d2lzh_tensorflow2 as d2l
print(tf.__version__)

2.3.0


## 3.9.1 获取和读取数据

这里继续使用Fashion-MNIST数据集。我们将使用多层感知机对图像进行分类。

In [3]:
from tensorflow.keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
batch_size = 256
x_train = tf.cast(x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)
x_train = x_train/255.0
x_test = x_test/255.0
train_iter = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_iter = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(batch_size)

4423680/4422102 [==============================] - 1s 0us/step


## 3.9.2 定义模型参数

我们在3.6节（softmax回归的从零开始实现）里已经介绍了，Fashion-MNIST数据集中图像形状为 $28 \times 28$，类别数为10。本节中我们依然使用长度为 $28 \times 28 = 784$ 的向量表示每一张图像。因此，输入个数为784，输出个数为10。实验中，我们设超参数隐藏单元个数为256。

In [7]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256
W1 = tf.Variable(tf.random.normal(shape=(num_inputs,num_hiddens),mean=0,stddev=0.01,dtype=tf.float32))
b1 = tf.Variable(tf.zeros(num_hiddens, dtype=tf.float32))
W2 = tf.Variable(tf.random.normal(shape=(num_hiddens, num_outputs),mean=0, stddev=0.01, dtype=tf.float32))
b2 = tf.Variable(tf.random.normal([num_outputs], stddev=0.1))

## 3.9.3 定义激活函数

这里我们使用基础的`max`函数来实现ReLU，而非直接调用`relu`函数。

In [6]:
def relu(x):
    return tf.math.maximum(x,0)

## 3.9.4 定义模型

同softmax回归一样，我们通过`reshape`函数将每张原始图像改成长度为`num_inputs`的向量。然后我们实现上一节中多层感知机的计算表达式。

In [12]:
def net(x):
    x = tf.reshape(x, shape=[-1,num_inputs])
    h = relu(tf.matmul(x,W1) + b1)  #从输入层到隐层,还使用了relu
    return tf.math.softmax(tf.matmul(h,W2) + b2) # 从隐层到输出层，同时用上了 sofamax

## 3.9.5 定义损失函数

为了得到更好的数值稳定性，我们直接使用Tensorflow提供的包括**softmax运算**和**交叉熵损失计算**的函数。

In [10]:
def loss(y_hat, y_true):
    return tf.losses.sparse_categorical_crossentropy(y_true,y_hat)

## 3.9.6 训练模型

训练多层感知机的步骤和3.6节中训练softmax回归的步骤没什么区别。我们直接调用`d2l`包中的`train_ch3`函数，它的实现已经在3.6节里介绍过。我们在这里设超参数迭代周期数为5，学习率为0.5。

In [13]:
num_epochs, lr = 5, 0.5
params = [W1, b1, W2, b2]
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params, lr)

epoch 1, loss 0.7961, train acc 0.702, test acc 0.790
epoch 2, loss 0.4830, train acc 0.820, test acc 0.835
epoch 3, loss 0.4213, train acc 0.844, test acc 0.851
epoch 4, loss 0.3857, train acc 0.857, test acc 0.857
epoch 5, loss 0.3647, train acc 0.864, test acc 0.862


## 小结

* 可以通过手动定义模型及其参数来实现简单的多层感知机。
* 当多层感知机的层数较多时，本节的实现方法会显得较烦琐，例如在定义模型参数的时候。

-----------

> 注：本节除了代码之外与原书基本相同，[原书传送门](https://zh.d2l.ai/chapter_deep-learning-basics/mlp-scratch.html)

